In [30]:
import spacy
import re
from gensim.models import Doc2Vec
from numpy import dot
from numpy.linalg import norm
from time import time

In [2]:
nlp = spacy.load('de_core_news_md', disable=['ner', 'parser'])

In [29]:
model = Doc2Vec.load("model.model")

In [24]:
def lemma(doc):
    txt = [token.lemma_ for token in doc if not token.is_stop]
    if len(txt) > 2:
        return ' '.join(txt)
def clean_sentences(sentences):
    tmp_sentences = [ re.sub("[^A-ZAAÖUa-zäöü#+']+", ' ', sentence).lower() for sentence in sentences]
    return [lemma(doc) for doc in nlp.pipe(tmp_sentences, batch_size=5000, n_threads=-1)]
    

In [42]:
sent1='''
Für einen unserer Kunden suchen wir derzeit einen freiberuflichen Mitarbeiter, der die nachfolgenden Anforderungen erfüllt:

Projektbeschreibung:
Software Entwicklung im Rahmen einer Weiterentwicklung eines Odoo ERP samt Schnittstellen-Systemen in Kubernetes.

Ihre Aufgaben:
Betrieb und Weiterentwicklung einer Netzwerkarchitektur von ESB Routen mit Apache Camel
Projektbezogene Softwareprogrammierung
Weiterentwicklung des ERP Systems

Ihr Profil:
gute Erfahrung im Umgang mit XLST, XML, und Kubernetes
gute bis sehr gute Erfahrung in der Java Programmierung (8)
gute Kenntnisse in der Python und Groovy Programmierung
Erfahrung im Umgang mit Build-Management-Tools (Maven und Kaniko) sowie Build und Deployment Pipelines
Erfahrung im Handel und E-Commerce von Vorteil
sehr gute Deutschkenntnisse

Rahmenparameter:
Einsatzort: Hamburg
Dauer: asap - 3 Monate
Auslastung: 4-5 Tage vor Ort
'''

In [43]:
sent2='''
Tätigkeit:

Projekt-/Aufgabenbeschreibung:
• Programmierung / Implementierung / Customizing technischer Komponenten in Applikationen auf Basis der genutzten Technologien inkl. dazugehöriger Schnittstellen, Module und entsprechender Datenbanken unter Einhaltung der Standards des Auftraggebers (z.B. Nutzung der standardisierten Entwicklungsumgebungen und Programmierrichtlinien)
• Beheben von Fehlern aus den Testphasen in den Applikationen / Komponenten
• Dokumentation der technischen Komponenten
• Erstellen von Systemdokumentationen
• Erstellen von Lieferpaketen für die betreffenden Applikationen
• Einrichten von Entwicklungsumgebungen incl. Continuous Integration
• Durchführen von Code-Reviews, automatisierten Unit-Tests
• Vorbereitung von Entscheidungsvorlagen zu technischen Themen


Anforderungen:

must have:
• Mobile Software Entwicklung: Erfahrung in der SW-Implementierung im Bereich Android (Erfahrung in der Entwicklung mobiler, nativer Anwendungen)
• Aktuelle Kenntnisse und Erfahrungen in der Anwendung folgender Technologien der Softwareentwicklung: Java, App-Entwicklung(Android), GIT, Gradle
• Agile Methodenkenntnisse: Praktische Erfahrung in Methoden und Vorgehen nach SCRUM und sonstigen agilen Vorgehensweisen
• Projektreferenzen zu o.g. Technologieschwerpunkten

nice to have:
• Teamarbeit und weitgehend selbstständiges Arbeiten an Aufgabenpaketen: Erfahrung in der Arbeit im Umfeld von mittleren Entwicklerteams, SW-Entwicklung im Pair
• Architekturkenntnisse: Aktuelle Kenntnisse der Softwarearchitektur, besonders im Kontext des Designs und Entwicklung von mobilen Apps.
• Sprachkenntnisse: Gute Sprachkenntnisse in Wort und Schrift in Polnisch und Deutsch
• Zertifizierungen zu o.g. Technologieschwerpunkten
• Zertifizierung als Certified Scrum Developer (CSD) oder vergleichbar
'''

In [44]:
t = time()
clean_sents=clean_sentences([sent1, sent2])
print('Time to clean up everything: {} seconds'.format(time() - t))
t= time()
a = model.infer_vector(clean_sents[0].split(),steps=500,alpha=0.025)
b = model.infer_vector(clean_sents[1].split(),steps=500,alpha=0.025)
print('Time to infer vector: {} seconds'.format(time() - t))
t= time()
cos_sim = dot(a, b)/(norm(a)*norm(b))
print(cos_sim)
print('Time to calculate cosine similarity: {} seconds'.format(time() - t))

Time to clean up everything: 0.040990352630615234 seconds
Time to infer vector: 0.14799952507019043 seconds
0.60132027
Time to calculate cosine similarity: 0.0 seconds


In [45]:
clean_sents

['  kunden suchen derzeit freiberuflich mitarbeiter nachfolgend anforderungen erfüllen projektbeschreibung software entwicklung rahmen weiterentwicklung odoo erp samt schnittstellen systemen kubernetes aufgaben betrieb weiterentwicklung netzwerkarchitektur esb routen apache camel projektbezogene softwareprogrammierung weiterentwicklung erp systems profil erfahrung umgang xlst xml kubernetes erfahrung java programmierung kenntnisse python groovy programmierung erfahrung umgang build management tools maven kaniko build deployment pipelines erfahrung handel e commerce vorteil deutschkenntnisse rahmenparameter einsatzort hamburg dauer asap monate auslastung ort',
 '  tätigkeit projekt aufgabenbeschreibung programmierung implementierung customizing technisch komponenten applikationen basis genutzt technologien inkl dazugehörig schnittstellen module entsprechend datenbanken einhaltung standards auftraggebers z b nutzung standardisieren entwicklungsumgebungen programmierrichtlinien beheben fe